# Ergodic Information Harvesting Simulation
This is the centralized code for you to run all of the simultions necessary to reproduce the raw data for the paper.

## 1. Install and import dependencies

In [ ]:
# Run this block if you have any missing python dependencies
!pip install scipy==1.0.1 numpy==1.14.3

In [ ]:
from itertools import product 
from multiprocessing import Pool, cpu_count
from scipy.io import loadmat, savemat
from numpy import linspace, arange, concatenate
from os import makedirs, scandir
from os.path import exists
from shutil import copy2
import numpy as np
# Suppress all warnings
np.seterr(all='ignore')
import warnings
warnings.filterwarnings("ignore")
# UTC Timestamp
import time
from datetime import datetime
# Ergodic Harvesting Code
from ErgodicHarvestingLib.SimParameters import ErgodicParameters, EIDParameters
from ErgodicHarvestingLib.Simulation import EIDSim
from SimulationMain import SimulationMain

## 2. Specify simulation target
**This is the place for you to input the specifications.** Use this section to specify the target figure you want to simulate. The available parameters for `targetFigure` is listed below:

- `fig1` - Simulate ergodic harvesting and infotaxis data used for figure 1
- `fig2` - Simulate ergodic harvesting data for electric fish, rat, and mole under strong and weak signal in figure 2
- `fig3` - Simulate infotaxis data for figure 3 panel a and b (used in the caption). Note that the ergodic harvesting data used for panel a and b are the same for figure 2 and thus omitted to avoid duplicated simulation jobs. The ergodic harvesting and infotaxis data used for panel c is provided by `sm-fig1` simulation
- `sm-fig1` - Simulate a large samples of ergodic harvesting and infotaxis trials for supplementary figure 1. The same data is also used for panel c of figure 3 in the main paper.
- `sm-fig5` - Simulate ergodic harvesting data used in supplement figure 5

The simulation is accelerated using parallel computing that leverage multiple CPU thread to simultaneously simulate multiple trails. Use `nThread` to specify the maximum number of thread available for the simulation and the code will use **up to** `nThread` if needed. If you want to achieve maximum speed, consider use `nThread = cpu_count()` or follow the included tutorial about how to use public computing cloud like Google Cloud or AWS to accelerate.

In [ ]:
targetFigure = ['fig1', 'fig2', 'fig3', 'sm-fig1', 'sm-fig5']
nThread = 16

## 3. Submit simulations
The following code find all of the simulations needed to run and submit them in a batch fashion.

In [ ]:
timeStampStart0 = datetime.fromtimestamp(time.time())
for fig in targetFigure:
    timeStampStart = datetime.fromtimestamp(time.time())
    # Find parameter files
    paramPath = './FigParameters/{0}/'.format(fig)
    params = [f.name for f in scandir(paramPath) if f.is_file() and '.json' in f.name]

    # Launch simulation for current figure
    nSimJobs = len(params)
    jobIdx = 1
    for p in params:
        print('Submitting simulation job - {0}\n'
              'Progress {1}/{2} ({3}%)'.format(p.replace('.json',''), jobIdx, nSimJobs, 100.0*jobIdx/nSimJobs))
        print('---------------------------------------------------------------')
        SimulationMain(paramPath+p, nThread=nThread)
        jobIdx += 1

    timeStampEnd = datetime.fromtimestamp(time.time())
    timeString = timeStampEnd.strftime("%b-%d-%Y %T")
    durationSecond = (timeStampEnd - timeStampStart).total_seconds()
    print("Figure {0} simulation finished at {1}, total time taken for simulation {2} seconds".format(fig, timeString, durationSecond))

durationSecond = (timeStampEnd - timeStampStart0).total_seconds()
print("All done! EOF at {0}, total time taken for all simulation(s) {1}".format(timeString, durationSecond))